# **1 기본적인 Xpath 사용법**
## **01 Initialized**
- XPath 는 `query language` 로써 XML의 nodes 들을 선택하는데 용이하다.
- [참고 Jupyter Notebook](http://nbviewer.jupyter.org/github/TwistedHardware/mltutorial/blob/master/notebooks/quick_tips/4.%20XPath.ipynb)

In [6]:
html_code = """
<html>
  <body>
    <h1>Favorite Python Librarires</h1>
    <ul>
      <li>Numpy</li>
      <li>Pandas</li>
      <li>requests</li>
    </ul>
  </body>
</html>"""

import requests
from lxml.html import tostring, fromstring, HTMLParser
html_code = html_code.replace("\\n","\n")
html_code

'\n<html>\n  <body>\n    <h1>Favorite Python Librarires</h1>\n    <ul>\n      <li>Numpy</li>\n      <li>Pandas</li>\n      <li>requests</li>\n    </ul>\n  </body>\n</html>'

In [5]:
# XPath 분석 가능한 객체로 변환
doc = fromstring(html_code)
doc

<Element html at 0x7f3d28211720>

## **02 Xpath to find nodes in a document**
- 대상 객체가 1개일 때
- 대상 객체가 여럿일 때

In [9]:
# .text : 대상객체의 text 내용
# .tag : 대상객체의 html 태그
# .attrib : 대상객체의 속성내용 dict()
title = doc.xpath("/html/body/h1")[0]
title, title.text, title.tag, title.attrib

(<Element h1 at 0x7f3d2853b400>, 'Favorite Python Librarires', 'h1', {})

In [10]:
title = doc.xpath("/html/body/h1/text()")[0]
title

'Favorite Python Librarires'

In [11]:
# Xpath와 일치하는 객체가 3개 발견되었다.
item_list = doc.xpath("/html/body/ul/li")
item_list

[<Element li at 0x7f2e60d378b8>,
 <Element li at 0x7f2e60d37908>,
 <Element li at 0x7f2e60d37958>]

In [12]:
# 호출 가능한 3개 객체를 `text()` 문법으로 호출하기
doc = fromstring(html_code)
item_list = doc.xpath("/html/body/ul/li/text()")
item_list

['넘파이 Numpy', '판다스 Pandas', '리퀘스트 requests']

## **03 Tag selector 를 활용한 크롤링 (모든 경로를 활용하지 않는다)**
Double forward slash `//` : 문서 어디에 있는지를 구분하지 않고, 일치하는 객체를 크롤링 한다 

In [11]:
doc = fromstring(html_code)
item_list = doc.xpath("//li/text()")  # "/html/body/ul/li/text()" 를 생략한 일부분의 입력만으로 출력가능
item_list

['Numpy', 'Pandas', 'requests']

## **04 Attributes selector**
두개의 `<h1>` tags 에 각기 다른 css 가 정의된 객체들을 구별 가능하다

In [17]:
html_code = """
<html>
  <body>
    <h1 class="text-muted">내가 가장 선호하는 라이브러리 Favorite Python Librarires</h1>
    <ul class="nav nav-pills nav-stacked">
      <li role="presentation"><a href="http://www.numpy.org/">넘파이 Numpy</a></li>
      <li role="presentation"><a href="http://pandas.pydata.org/">판다스 Pandas</a></li>
      <li role="presentation"><a href="http://python-requests.org/">리퀘스트 requests</a></li>
    </ul>
    <h1 class="text-success">Favorite JS Librarires</h1>
    <ul class="nav nav-tabs">
      <li role="presentation"><a href="http://getbootstrap.com/">부트스트랩 Bootstrap</a></li>
      <li role="presentation"><a href="https://jquery.com/">제이쿼리 jQuery</a></li>
      <li role="presentation"><a href="http://d3js.org/">d3.js</a></li>
    </ul>
</html>"""

html_code = html_code.replace("\\n","\n")
doc = fromstring(html_code)
doc

<Element html at 0x7f3d2853b860>

In [18]:
title = doc.xpath("/html/body/h1[@class='text-muted']/text()")[0]
title

'내가 가장 선호하는 라이브러리 Favorite Python Librarires'

## **2 `contains()` 함수 활용하기**
- Xpath 내부의 [index] 리스트 문법에서, [ index ] 내부에 `contains()`함수를 활용하면
"full class for selection" 또는 "one of the classed only used"를 사용 가능하다 

In [19]:
item_list = doc.xpath("/html/body/ul[contains(@class,'nav-stacked')]/li/a/text()")
item_list

['넘파이 Numpy', '판다스 Pandas', '리퀘스트 requests']

## 2. Returning attributes

태그 내부의 '속성'값 (ex)`<a>` 태그 `href`속성의 ...link..값) 만 호출가능

/text() 함수 없이도 value만 호출된다

In [20]:
item_list = doc.xpath("/html/body/ul[contains(@class,'nav-stacked')]/li/a/@href")
item_list

['http://www.numpy.org/',
 'http://pandas.pydata.org/',
 'http://python-requests.org/']

In [21]:
# find_population_lxml.py
import re
from lxml import html
broken_html = '<ul class=country><li>Area<li>Population</ul>'
tree = html.fromstring(broken_html)  # HTML 분석
fixed_html = html.tostring(tree, pretty_print=True); fixed_html.decode('utf-8')

'<ul class="country">\n<li>Area</li>\n<li>Population</li>\n</ul>\n'

In [22]:
from lxml import html
import requests
url = 'http://media.daum.net/ranking/empathy'
page = requests.get(url)
tree = html.fromstring(page.content)
page.text[:300]

'\n<!DOCTYPE html>\n\n\n\n<html lang="ko" class="os_unknown none unknown version_0 ">\n<head>\n<meta charset="utf-8">\n<meta name="referrer" content="always" />\n\n<meta property="og:author" content="Daum 뉴스" />\n<meta property="og:site_name" content="다음뉴스" />\n<meta property="og:title" content="홈"/>\n<meta prope'

In [25]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
hotel_name

[]

In [24]:
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

IndexError: list index out of range

In [ ]:
vv/home/buffet/Coding/Gits/PyFinance/Server/SQL.ipynb
/home/buffet/Coding/Gits/PyFinance/Server/SQL_trader.ipynb